In [ ]:
#pip install elasticsearch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 385 kB 6.0 MB/s 
     |████████████████████████████████| 59 kB 4.2 MB/s 
     |████████████████████████████████| 140 kB 49.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.13 which is incompatible.


# Módulos

In [ ]:
# Importacion de modulos 
import elasticsearch
import numpy as np
from elasticsearch import Elasticsearch
from elasticsearch import client
from datetime import datetime
from time import sleep

# Inputs 

In [ ]:
# Informacion sobre la incidencia
fecha_inicio_incidencia = "2022-03-04 23:50:00" # Meter la hora según timezone
fecha_fin_incidencia    = "2022-03-06 00:10:00" # Meter la hora según timezone

# Nombre del Job de Elastic
jobname                 = "sabado_anomalia"

In [ ]:
# Datos de la conexion con el servidor de Elastic
ELASTIC_PASSWORD = "9QJVpAsI1dfU33vZpc072VTd"
CLOUD_ID = "SeriestemporalesElastic:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyQxMmRiODkwZjhhN2U0YTVjOTAwMDViODMyZjhlYWViNCRjMmU4YmQxYjkzZTI0MGY5OTZiYzIyNTFkZGMxMGY4Yg=="

client = Elasticsearch(
    cloud_id=CLOUD_ID,
    basic_auth=("elastic", ELASTIC_PASSWORD)
)

# Nuevas Variables

In [ ]:
# Calculo del offset por cambio de hora

# Timestamps fechas incidencia
fecha_inicio_incidencia = datetime.strptime(fecha_inicio_incidencia, '%Y-%m-%d %H:%M:%S')
fecha_inicio_incidencia_timestamp = datetime.timestamp(fecha_inicio_incidencia)*1000

fecha_fin_incidencia = datetime.strptime(fecha_fin_incidencia, '%Y-%m-%d %H:%M:%S')
fecha_fin_incidencia_timestamp = datetime.timestamp(fecha_fin_incidencia)*1000

# Timestamps de las fechas de cambio de hora

# Primer cambio de hora
fecha_cambio_hora_1 = "2022-03-27 02:00:00" 
fecha_cambio_hora_1 = datetime.strptime(fecha_cambio_hora_1, '%Y-%m-%d %H:%M:%S')
fecha_cambio_hora_1_timestamp = datetime.timestamp(fecha_cambio_hora_1)*1000

# Segundo cambio de hora
fecha_cambio_hora_2 = "2022-10-30 02:00:00" 
fecha_cambio_hora_2 = datetime.strptime(fecha_cambio_hora_2, '%Y-%m-%d %H:%M:%S')
fecha_cambio_hora_2_timestamp = datetime.timestamp(fecha_cambio_hora_2)*1000

# Comparamos fechas en el mismo año 
fecha_inicio_incidencia_cambio_timestamp = (datetime(2022, fecha_inicio_incidencia.month, fecha_inicio_incidencia.day, fecha_inicio_incidencia.hour, fecha_inicio_incidencia.minute, fecha_inicio_incidencia.second).timestamp())*1000 

if (fecha_inicio_incidencia_cambio_timestamp < fecha_cambio_hora_1_timestamp):
    offset_1 = -3600 

if ((fecha_inicio_incidencia_cambio_timestamp > fecha_cambio_hora_1_timestamp) and (fecha_inicio_incidencia_cambio_timestamp < fecha_cambio_hora_2_timestamp)):
  offset_1 = -7200

if (fecha_cambio_hora_2_timestamp < fecha_inicio_incidencia_cambio_timestamp):
  offset_1 = -3600  

fecha_fin_incidencia_cambio_timestamp = (datetime(2022, fecha_fin_incidencia.month, fecha_fin_incidencia.day, fecha_fin_incidencia.hour, fecha_fin_incidencia.minute, fecha_fin_incidencia.second).timestamp())*1000 

if (fecha_fin_incidencia_cambio_timestamp < fecha_cambio_hora_1_timestamp):
    offset_2 = -3600 

if ((fecha_fin_incidencia_cambio_timestamp > fecha_cambio_hora_1_timestamp) and (fecha_fin_incidencia_cambio_timestamp < fecha_cambio_hora_2_timestamp)):
  offset_2 = -7200

if (fecha_cambio_hora_2_timestamp < fecha_fin_incidencia_cambio_timestamp):
  offset_2 = -3600  

In [ ]:
# Transformacion de variables de entrada
# Fechas
fecha_inicio_incidencia = str((int(fecha_inicio_incidencia_timestamp + offset_1*1000)))
fecha_fin_incidencia = str((int(fecha_fin_incidencia_timestamp + offset_2*1000)))

# Nombre del Datafeed
datafeed_id = "datafeed-" + jobname

# Cliente de ML
cliente_ml = client.ml

# Solución

In [ ]:
# Paso 1. Parar y cerrar el Job
cliente_ml.close_job(job_id=jobname)

ObjectApiResponse({'closed': True})

In [ ]:
# Paso 2. Escoger el id de la snapshot con latest_record_time_stamp mas cercano a la incidencia
snapshots_available = cliente_ml.get_model_snapshots(job_id=jobname)
# Recoger el count 
number_snapshots = snapshots_available["count"]
# Iterar guardando en una matriz el id del snpashot y latest_record_time_stamp
snapshots_info = snapshots_available["model_snapshots"]
latest = np.zeros((number_snapshots, 2))
for i in range(number_snapshots):
  latest[i,0] = snapshots_info[i]["latest_record_time_stamp"]

for i in range(number_snapshots):
  latest[i,1] = snapshots_info[i]["snapshot_id"]

# Comparar con la fecha de la incidencia para buscar cual esta inmediatamente antes
  # Ordenar por timestamp (primera columna)
  latest1 = latest[latest[:,0].argsort()]
  latest2 = np.flip(latest1, axis=0)
  # Eliminar valores mayores
  latest3 = latest2[latest2[:,0]<float(fecha_inicio_incidencia)]
# Escoger el id de la reversion correcta
id_revert = latest3[0,1]
id_revert = str(int(id_revert))

In [ ]:
# Paso 3. Revertir el Job antes de la incidencia 
cliente_ml.revert_model_snapshot(job_id=jobname, snapshot_id=id_revert, delete_intervening_results=True)
sleep(5)

In [ ]:
# Paso 4. Abrir el Job 
cliente_ml.open_job(job_id=jobname)
sleep(5)

In [ ]:
# Paso 5. Avanzamos datafeed hasta la fecha de inicio de incidencia
cliente_ml.start_datafeed(datafeed_id=datafeed_id, end=fecha_inicio_incidencia)
state_datafeed = 1

# Una vez completado el paso continuamos al siguiente
while (state_datafeed == 1):
  sleep(10)
  state_datafd = cliente_ml.get_datafeed_stats(datafeed_id=datafeed_id)
  st = state_datafd['datafeeds']
  state_actual = st[0]['state'] 
  if (state_actual == 'stopped'):
    state_datafeed = 0

In [ ]:
# Paso 6. Abrir el Job 
cliente_ml.open_job(job_id=jobname)
sleep(5)

In [ ]:
# Paso 7. Avanzamos datafeed desde la fecha de fin de incidencia hasta la fecha actual
cliente_ml.start_datafeed(datafeed_id=datafeed_id, start=str(int(fecha_fin_incidencia)))

ObjectApiResponse({'started': True, 'node': '1UbOBFWrRuiIttmnruod0A'})

# Parte de Indices

In [4]:
#!pip install eland
#!pip install elasticsearch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 143 kB 5.0 MB/s 
     |████████████████████████████████| 385 kB 35.1 MB/s 
     |████████████████████████████████| 59 kB 4.2 MB/s 
     |████████████████████████████████| 140 kB 58.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.13 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
# Importacion de modulos 
import elasticsearch
import numpy as np
from elasticsearch import Elasticsearch
from elasticsearch import client
from datetime import datetime
from time import sleep
from elasticsearch.client import IndicesClient
import eland as ed
import pandas as pd
from elasticsearch.client import IngestClient

# Inputs

In [6]:
# Informacion sobre la incidencia
fecha_inicio_incidencia = "2022-03-04 23:50:00" # Meter la hora segun timezone, no en UTC
fecha_fin_incidencia    = "2022-03-06 00:10:00" # Meter la hora según timezone, no en UTC

# Nombre del Job de Elastic
jobname                 = "sabado_anomalia"
variable_name_predict   = "y_an"

# Nombre del indice sobre el que se realiza el Job
index_data              = 'sabado_incidencia'

# Nuevas variables

In [7]:
# Calculo del offset por cambio de hora

# Timestamps fechas incidencia
fecha_inicio_incidencia = datetime.strptime(fecha_inicio_incidencia, '%Y-%m-%d %H:%M:%S')
fecha_inicio_incidencia_timestamp = datetime.timestamp(fecha_inicio_incidencia)*1000

fecha_fin_incidencia = datetime.strptime(fecha_fin_incidencia, '%Y-%m-%d %H:%M:%S')
fecha_fin_incidencia_timestamp = datetime.timestamp(fecha_fin_incidencia)*1000

# Timestamps de las fechas de cambio de hora

# Primer cambio de hora
fecha_cambio_hora_1 = "2022-03-27 02:00:00" 
fecha_cambio_hora_1 = datetime.strptime(fecha_cambio_hora_1, '%Y-%m-%d %H:%M:%S')
fecha_cambio_hora_1_timestamp = datetime.timestamp(fecha_cambio_hora_1)*1000

# Segundo cambio de hora
fecha_cambio_hora_2 = "2022-10-30 02:00:00" 
fecha_cambio_hora_2 = datetime.strptime(fecha_cambio_hora_2, '%Y-%m-%d %H:%M:%S')
fecha_cambio_hora_2_timestamp = datetime.timestamp(fecha_cambio_hora_2)*1000

# Comparamos fechas en el mismo año 
fecha_inicio_incidencia_cambio_timestamp = (datetime(2022, fecha_inicio_incidencia.month, fecha_inicio_incidencia.day, fecha_inicio_incidencia.hour, fecha_inicio_incidencia.minute, fecha_inicio_incidencia.second).timestamp())*1000 

if (fecha_inicio_incidencia_cambio_timestamp < fecha_cambio_hora_1_timestamp):
    offset_1 = -3600 

if ((fecha_inicio_incidencia_cambio_timestamp > fecha_cambio_hora_1_timestamp) and (fecha_inicio_incidencia_cambio_timestamp < fecha_cambio_hora_2_timestamp)):
  offset_1 = -7200

if (fecha_cambio_hora_2_timestamp < fecha_inicio_incidencia_cambio_timestamp):
  offset_1 = -3600  

fecha_fin_incidencia_cambio_timestamp = (datetime(2022, fecha_fin_incidencia.month, fecha_fin_incidencia.day, fecha_fin_incidencia.hour, fecha_fin_incidencia.minute, fecha_fin_incidencia.second).timestamp())*1000 

if (fecha_fin_incidencia_cambio_timestamp < fecha_cambio_hora_1_timestamp):
    offset_2 = -3600 

if ((fecha_fin_incidencia_cambio_timestamp > fecha_cambio_hora_1_timestamp) and (fecha_fin_incidencia_cambio_timestamp < fecha_cambio_hora_2_timestamp)):
  offset_2 = -7200

if (fecha_cambio_hora_2_timestamp < fecha_fin_incidencia_cambio_timestamp):
  offset_2 = -3600  

In [8]:
# Transformacion de variables de entrada
# Fechas
fecha_inicio_incidencia = (fecha_inicio_incidencia_timestamp + offset_1*1000)/1000
fecha_fin_incidencia    = (fecha_fin_incidencia_timestamp + offset_2*1000)/1000

fecha_inicio_incidencia_datetime = datetime.fromtimestamp(fecha_inicio_incidencia)
fecha_fin_incidencia_datetime = datetime.fromtimestamp(fecha_fin_incidencia)

# Nombre del Datafeed
datafeed_id = "datafeed-" + jobname

In [ ]:
fecha_inicio_incidencia_datetime

datetime.datetime(2022, 3, 4, 22, 50)

In [10]:
# Datos de la conexion con el servidor de Elastic
ELASTIC_PASSWORD = "9QJVpAsI1dfU33vZpc072VTd"
CLOUD_ID = "SeriestemporalesElastic:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyQxMmRiODkwZjhhN2U0YTVjOTAwMDViODMyZjhlYWViNCRjMmU4YmQxYjkzZTI0MGY5OTZiYzIyNTFkZGMxMGY4Yg=="

es_client = Elasticsearch(
    cloud_id=CLOUD_ID,
    basic_auth=("elastic", ELASTIC_PASSWORD)
)
cliente_index = IndicesClient(client=es_client)

In [11]:
# Creacion del Indice de Backup
indexname = 'indicebackup'
try:
  cliente_index.create(index=indexname)
except:
  cliente_index.delete(index=indexname)
  cliente_index.create(index=indexname)

In [ ]:
# Reindexar datos de la anomalia al indice backup (me queda hacer el filtro de solo el tiempo de anomalia)
cliente_index.reindex(source=index_data, dest=indexname)

AttributeError: ignored

In [12]:
# Obtener datos de la prediccion del Job para el tiempo de anomalia
df = ed.DataFrame(es_client, es_index_pattern=".ml-anomalies-shared")
# Filtramos para nuestro job de interes
df = df[df["job_id"] == jobname]
# Obtenemos las columnas de interes del dataframe
df_select = df[["timestamp", "model_median", "model_upper"]]
# Eliminar filas con valores nulos
df_select = ed.eland_to_pandas(df_select)
df_select = df_select.dropna()
df_select.head(20)
# Ordenamos filas por timestamp
df_select.timestamp
# Seleccionamos timestamp de interes
df_select.head(20)

,timestamp,model_median,model_upper
sabado_anomalia_model_plot_1641108600000_600_0_0_0,2022-01-02 07:30:00,730.720582,970.417156
sabado_anomalia_model_plot_1641109200000_600_0_0_0,2022-01-02 07:40:00,733.142097,980.683989
sabado_anomalia_model_plot_1641109800000_600_0_0_0,2022-01-02 07:50:00,737.074255,981.281696
sabado_anomalia_model_plot_1641110400000_600_0_0_0,2022-01-02 08:00:00,746.954375,948.065246
sabado_anomalia_model_plot_1641111000000_600_0_0_0,2022-01-02 08:10:00,745.925155,948.629392
sabado_anomalia_model_plot_1641111600000_600_0_0_0,2022-01-02 08:20:00,744.179300,949.583249
sabado_anomalia_model_plot_1641112200000_600_0_0_0,2022-01-02 08:30:00,741.907749,950.261045
sabado_anomalia_model_plot_1641112800000_600_0_0_0,2022-01-02 08:40:00,739.130187,950.595557
sabado_anomalia_model_plot_1641113400000_600_0_0_0,2022-01-02 08:50:00,735.833594,950.685110
sabado_anomalia_model_plot_1641114000000_600_0_0_0,2022-01-02 09:00:00,732.048903,950.408468


In [13]:
df_select['timestamp'] = pd.to_datetime(df_select.timestamp)

In [14]:
df_select_order = df_select.sort_values(by="timestamp", ascending=True)

In [15]:
df_select_order = df_select_order.set_index("timestamp")
df_select_order.head(20)

,model_median,model_upper
timestamp,,
2021-12-31 23:00:00,0.000000,0.000000
2021-12-31 23:10:00,0.000000,0.000000
2021-12-31 23:20:00,0.000000,0.000000
2021-12-31 23:30:00,0.000000,0.000000
2021-12-31 23:40:00,0.000000,0.000000
2021-12-31 23:50:00,0.000000,0.000000
2022-01-01 00:00:00,0.000000,0.000000
2022-01-01 00:10:00,0.000000,0.000000
2022-01-01 00:20:00,0.000000,0.000000


In [16]:
df_select_filter = df_select_order.loc[fecha_inicio_incidencia_datetime:fecha_fin_incidencia_datetime]

In [18]:
# Hacer un rename del nombre de columna "model_median" al nombre de la variable que se predice en el JOB
df_select_filter.rename(columns = {'model_median':variable_name_predict}, inplace = True)

# Eliminar variable upper_name
df_select_filter = df_select_filter.drop('model_upper', axis=1)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [23]:
# Crear la pipeline de ingesta para @timestamp y parsear a un indice los datos teniendo cuidado con el timestamp
df_select_filter = df_select_filter.reset_index()

client_Ingest = IngestClient(client=es_client)

In [25]:
# Eliminar del indice principal datos para el timestamp de la incidencia
client_Ingest.put_pipeline(id='procesador_date_anomalias', body={
    "date": {
      "field": "timestamp",
      "formats": [
        "UNIX_MS"
      ],
      "target_field": "@timestamp"
    }
  })

#[
#  {
#    "date": {
#      "field": "timestamp",
#      "formats": [
#        "UNIX_MS"
#      ],
#      "target_field": "@timestamp"
#    }
#  }
#]

client_Ingest.put_pipeline(id='procesador_date_anomalias', body={
    'description': "Extract attachment information",
    'processors': [
        {"attachment": {"field": "cv"}}
    ]
})

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  


TypeError: ignored

In [ ]:
# Introducir nuevos documentos en este indice

In [ ]:
# Resetear Job desde la snapshot (buscar snapshot como en el modelo anterior y correr job)

In [ ]:
# Sustituir datos del indice backup al indice original para el tiempo de incidencia